<a href="https://colab.research.google.com/github/nehaMhr/Business-Contract-Validation/blob/main/Business_Contract_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
from google.colab import drive
drive.mount('/content/drive')

dataset_path = r"/content/drive/MyDrive/supplier-contracts.csv"
supplier_contracts = pd.read_csv(dataset_path, on_bad_lines='skip')
##supplier_contracts = datasets.load_iris()
X = supplier_contracts.data
y = supplier_contracts.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Print confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Mounted at /content/drive


AttributeError: 'DataFrame' object has no attribute 'data'

In [ ]:
pip install nltk spacy scikit-learn
python -m spacy download en_core_web_sm

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import numpy as np

# Load spaCy model for NLP
nlp = spacy.load('en_core_web_sm')

# Sample business contract template (for demonstration purposes)
contract_template = """
This Contract is made on [date] between [Party A] and [Party B]. The parties agree as follows:
1. Scope of Work: [Description of work].
2. Payment Terms: [Payment terms].
3. Confidentiality: [Confidentiality terms].
4. Term and Termination: [Term and termination terms].
5. Governing Law: [Governing law].
"""

# Deviations to be checked (sample deviations)
deviations = [
    "This Contract is made on [date] between [Party A] and [Party C].",
    "Payment Terms: [Different payment terms].",
    "Confidentiality: [Different confidentiality terms]."
]

# Function to preprocess text
def preprocess_text(text):
    # Remove stopwords and lemmatize
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

# Preprocess the contract template and deviations
processed_template = preprocess_text(contract_template)
processed_deviations = [preprocess_text(dev) for dev in deviations]

# Example data for training (using deviations as positive samples, and the template as negative sample)
data = [processed_template] * 3 + processed_deviations
labels = [0] * 3 + [1] * len(deviations)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

# Train a simple logistic regression model
model = LogisticRegression()
model.fit(X, labels)

# Function to highlight deviations in the contract
def highlight_deviations(contract, model, vectorizer):
    processed_contract = preprocess_text(contract)
    X_contract = vectorizer.transform([processed_contract])
    prediction = model.predict(X_contract)

    if prediction == 1:
        print("Deviation detected:")
        doc = nlp(contract)
        for sentence in doc.sents:
            processed_sentence = preprocess_text(sentence.text)
            X_sentence = vectorizer.transform([processed_sentence])
            if model.predict(X_sentence) == 1:
                print(f"-> {sentence.text.strip()}")
    else:
        print("No deviations detected.")

# Highlight deviations in the contract template
highlight_deviations(contract_template, model, vectorizer)